In [1]:
from typing import List
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import torch

In [2]:
model_path = r"/data/qwen2-7b-instruct"

In [3]:
tokenizer = AutoTokenizer.from_pretrained(
    model_path,
    trust_remote_code=True,
    local_files_only=True
)


In [4]:

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.float16,  # FP16
    device_map="auto",         # accelerate 自动分配 GPU
    trust_remote_code=True,
    local_files_only=True
)
model.eval()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 3584)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=3584, out_features=3584, bias=True)
          (k_proj): Linear(in_features=3584, out_features=512, bias=True)
          (v_proj): Linear(in_features=3584, out_features=512, bias=True)
          (o_proj): Linear(in_features=3584, out_features=3584, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (up_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (down_proj): Linear(in_features=18944, out_features=3584, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
      )
    )
    (norm):

In [21]:
# 这里假设使用零样本/少样本prompt方式，实际可根据具体模型调整
question = "有哪些销售额超过50000的店铺？"
prompt_template = None
"""
利用LLM和few-shot prompt抽取自然语言问题中的关键词。
"""
# 示例prompt，可根据实际模型和效果调整
if prompt_template is None:
    prompt_template = (
    "输入问题: {question}\n"
    "请严格遵循以下要求：\n"
    "1. 仅提取出问题中的关键实体、属性、数值或筛选条件，作为关键词；\n"
    "2. 仅输出以下标准 JSON 格式，不得添加任何额外解释或文字；\n"
    "3. 如果无法提取关键词或问题不清晰，请返回空列表。\n\n"
    "标准输出格式：\n"
    "{{\n"
    '  "keywords": ["<keyword1>", "<keyword2>", "..."]\n'
    "}}\n"
    "【特别注意】：\n"
    "- 不要输出除 JSON 之外的任何内容\n"
    "- 保持关键词简单精炼，如必要可保留数字和单位\n"
    "- 如果格式错误或输出多余内容，答案视为无效\n"
    )
prompt = prompt_template.format(question=question)
print(prompt)


输入问题: 有哪些销售额超过50000的店铺？
请严格遵循以下要求：
1. 仅提取出问题中的关键实体、属性、数值或筛选条件，作为关键词；
2. 仅输出以下标准 JSON 格式，不得添加任何额外解释或文字；
3. 如果无法提取关键词或问题不清晰，请返回空列表。

标准输出格式：
{
  "keywords": ["<keyword1>", "<keyword2>", "..."]
}
【特别注意】：
- 不要输出除 JSON 之外的任何内容
- 保持关键词简单精炼，如必要可保留数字和单位
- 如果格式错误或输出多余内容，答案视为无效



In [31]:
inputs = tokenizer(
    prompt,
    max_length=2048,
    truncation=True,            # 显式截断
    return_tensors="pt"
).to(model.device)

model.config.pad_token_id = tokenizer.pad_token_id  # 用专用 <pad>

out = model.generate(
    **inputs,
    max_new_tokens=256,
    temperature=0.7,
    top_p=0.9,
    return_dict_in_generate=True  # 关键！返回 structured outputs
)
# 只取 newly generated tokens
generated_tokens = out.sequences[:, inputs.input_ids.shape[1]:]  # 只保留新增部分
result = tokenizer.decode(generated_tokens[0], skip_special_tokens=True)
print(result)

```json
{
  "keywords": ["销售额", ">50000", "店铺"]
}
```


In [34]:
import re
cleaned = re.sub(r"^```(?:json)?\s*([\s\S]*?)\s*```$", r"\1", cleaned.strip())
print("cleaned: ", cleaned)
import json
try:
    parsed = json.loads(cleaned)
    keywords = parsed.get("keywords", [])
except json.JSONDecodeError:
    keywords = []
print(keywords)

cleaned:  {
  "keywords": ["销售额", ">50000", "店铺"]
}
['销售额', '>50000', '店铺']
